# This notebook is for Task 5 only

Improvements Task 5:
- Refer to whatsapp
- Composite of multi views/plots:
    + One is the height histogram 
    + One is the overall histogram (slidable)
    + Filter by dropdown country / region
    + Dropdown affects height and overall, overall also affects height
- Limit number of countries per region

In [5]:
# imports
import altair as alt
import pandas as pd
# altair specific
alt.data_transformers.disable_max_rows()
alt.data_transformers.enable('data_server')
alt.renderers.enable('mimetype')

# user defined modules
from helpers import preprocessing


In [6]:
# helper function
def get_data(data_path, preprocess=False):
    data = pd.read_csv(data_path)
    if preprocess is True:
        fifa_subset = preprocessing(data)
        america_subset = fifa_subset[fifa_subset["region"] == "America"]
        countries_counts = america_subset.country.value_counts().to_frame(name="count").reset_index().rename(columns={"index": "country"})
        countries_interests = countries_counts.query('count > 225').country.tolist()
        countries_subset = america_subset[america_subset.country.isin(countries_interests)]
        # store to data
        data = countries_subset
    return data

In [7]:
data_path = "data/player_raw_data.csv"
fifa = get_data(data_path)
subset = get_data(data_path, preprocess=True)